In [6]:
from random import seed, randrange
from math import sqrt
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

In [7]:
def load_wine_dataset():
    data = load_wine()
    dataset = []
    for i in range(len(data.data)):
        row = list(data.data[i]) + [data.target[i]]
        dataset.append(row)
    return dataset

In [8]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = []
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = []
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [9]:
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    accuracy_scores = []
    f1_scores = []
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = []
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_score(actual, predicted) * 100
        f1 = f1_score(actual, predicted, average='weighted') * 100
        accuracy_scores.append(accuracy)
        f1_scores.append(f1)
    return accuracy_scores, f1_scores

def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i])**2
    return sqrt(distance)

def find_bmu(codebooks, test_row):
    distances = []
    for codebook in codebooks:
        dist = euclidean_distance(codebook, test_row)
        distances.append((codebook, dist))
    distances.sort(key=lambda tup: tup[1])
    return distances[0][0]

def predict(codebooks, test_row):
    bmu = find_bmu(codebooks, test_row)
    return bmu[-1]

def random_codebook(train):
    n_records = len(train)
    n_features = len(train[0])
    codebook = [train[randrange(n_records)][i] for i in range(n_features)]
    return codebook

def train_codebooks(train, n_codebooks, lrate, epochs):
    codebooks = [random_codebook(train) for _ in range(n_codebooks)]
    for epoch in range(epochs):
        rate = lrate * (1.0 - (epoch / float(epochs)))
        for row in train:
            bmu = find_bmu(codebooks, row)
            for i in range(len(row) - 1):
                error = row[i] - bmu[i]
                if bmu[-1] == row[-1]:
                    bmu[i] += rate * error
                else:
                    bmu[i] -= rate * error
    return codebooks

# LVQ Algorithm
def learning_vector_quantization(train, test, n_codebooks, lrate, epochs):
    codebooks = train_codebooks(train, n_codebooks, lrate, epochs)
    predictions = []
    for row in test:
        output = predict(codebooks, row)
        predictions.append(output)
    return predictions

In [10]:
seed(42)

dataset = load_wine_dataset()

scaler = StandardScaler()
for i in range(len(dataset[0])-1):  # Standardize features (ignoring the label)
    col = [row[i] for row in dataset]
    scaled_col = scaler.fit_transform([[x] for x in col])
    for j in range(len(dataset)):
        dataset[j][i] = scaled_col[j][0]

n_folds = 5
learn_rate = 0.3
n_epochs = 50
n_codebooks = 20

accuracy_scores, f1_scores = evaluate_algorithm(dataset, learning_vector_quantization, n_folds, n_codebooks, learn_rate, n_epochs)
print('Accuracy Scores: %s' % accuracy_scores)
print('Mean Accuracy: %.3f%%' % (sum(accuracy_scores)/float(len(accuracy_scores))))
print('F1 Scores: %s' % f1_scores)
print('Mean F1 Score: %.3f' % (sum(f1_scores)/float(len(f1_scores))))

Accuracy Scores: [97.14285714285714, 100.0, 91.42857142857143, 91.42857142857143, 97.14285714285714]
Mean Accuracy: 95.429%
F1 Scores: [97.10808461315561, 100.0, 91.3647937935794, 91.3886113886114, 97.13497536945813]
Mean F1 Score: 95.399


En esta actividad, se implementan varias mejoras clave con respecto al código original de LVQ. Se cambió del conjunto de datos, escogiendo el dataset de vino de la biblioteca `sklearn.datasets`. El motivo de escoger este dataset proviene de la recomendación de buscar datasets de clasificación en repositorios como el de aprendizaje automático de UCI. Adicionalmente use`StandardScaler` para estandarizar las características antes de entrenar los modelos. El código original solo calculaba la precisión, pero en el mío se incluye el cálculo del F1 para mejor comprensión de las predicciones. Se mantuvieron la mayoría de las funciones clave, como la de encontrar la unidad de mejor coincidencia, entrenar los codebooks y realizar predicciones. A diferencia del notebook de SOM, decidí no explorar hiperparámetros para este problema debido a los buenos resultados obtenidos a través de la prueba y error. Aprendi que así como fue el caso para SOM, los dos se basan en encontrar el Best Matching Unit para optimizar las predicciones de los modelos. Este enfoque para resolver problemas de agrupación y de clasificaciones me brindaron un nuevo panorama de las alternativas que hay entre las redes neuronales densas y los metodos de aprendizaje automatico tradicionales. Ambos metodos brindan un nuevo lado por donde explorar para resolver este tipo de problemas.